In [2]:
### Validate allocation cycle
import os
import pandas as pd
import datetime as dt
import numpy as np
from src.config import ROOT_DIR, CONFIG

path_os = os.path.join(ROOT_DIR, 'data', 'output')
posto_id = '22'
process_id = '2881'
df_calendario = pd.read_csv(os.path.join(path_os, 'df_calendario-'+process_id+'-'+posto_id+'.csv'))
df_estimativas = pd.read_csv(os.path.join(path_os, 'df_estimativas-'+process_id+'-'+posto_id+'.csv'))
df_colaboradores = pd.read_csv(os.path.join(path_os, 'df_colaborador-'+process_id+'-'+posto_id+'.csv'))
df_insert_results = pd.read_csv(os.path.join(path_os, 'df_insert_results-'+process_id+'-'+posto_id+'.csv'))
df_results = pd.read_csv(os.path.join(path_os, 'df_results-'+process_id+'-'+posto_id+'.csv'))

data_ini = dt.datetime.strptime(CONFIG.get('external_call_data').get('start_date'), '%Y-%m-%d')
data_fim = dt.datetime.strptime(CONFIG.get('external_call_data').get('end_date'), '%Y-%m-%d')


## Totais de folgas aplicadas

In [ ]:
# Check how many L's we gave to each colaborador in the results

# Filter rows where horario is 'L' and count by colaborador
df_folgas_per_colab_L = df_results[df_results['horario'] == 'L'].groupby('colaborador').size().reset_index(name='L_count')
df_folgas_per_colab_LQ = df_results[df_results['horario'] == 'LQ'].groupby('colaborador').size().reset_index(name='LQ_count')
df_folgas_per

# Merge the two dataframes on the colaborador column
df_folgas_per_colab = pd.merge(df_folgas_per_colab_L, df_folgas_per_colab_LQ, on='colaborador', how='left')
df_folgas_per_colab['L_count'] = df_folgas_per_colab['L_count'].fillna(0)
df_folgas_per_colab['LQ_count'] = df_folgas_per_colab['LQ_count'].fillna(0)
print(df_folgas_per_colab)

# df_folgas_per_colab must have the same as in df_colaborador

    colaborador  L_count  LQ_count
0             1       98       5.0
1          5851       97       5.0
2          7656       89       5.0
3          7919       98       5.0
4      80000509       97       5.0
5      80000937       98       5.0
6      80000951       30       0.0
7      80001134       32       0.0
8      80001237       99       5.0
9      80001331       96       5.0
10     80001366       98       5.0
11     80001489       99       5.0
12     80001568       99       5.0
13     80001578       99       5.0
14     80001630       98       5.0
15     80003467       96       5.0


## Semanas de incumprimento

In [4]:
# Check the number of weeks in results that dont meet the maximum 5 days of work per week
df_results_copy = df_results.copy()
df_results_final = df_results.copy()

# Add a week number column
df_results_copy['data'] = pd.to_datetime(df_results_copy['data'])
df_results_final['data'] = pd.to_datetime(df_results_final['data'])
df_results_copy['week_number'] = df_results_copy['data'].dt.isocalendar().week
df_results_final['week_number'] = df_results_final['data'].dt.isocalendar().week

# Filter the data column to be between data_ini and data_fim
#print(df_colaboradores.columns)
df_colaborador = df_colaboradores.copy()
df_colaborador = df_colaborador[['matricula', 'data_admissao', 'data_demissao']]
df_colaborador['data_admissao'] = pd.to_datetime(df_colaborador['data_admissao'])
df_colaborador['data_demissao'] = df_colaborador['data_demissao'].replace([0, '0', np.nan], data_fim)
df_colaborador['data_demissao'] = pd.to_datetime(df_colaborador['data_demissao'])
df_results_copy = pd.merge(df_results_copy, df_colaborador, left_on='colaborador', right_on='matricula', how='left')
df_results_copy = df_results_copy[(df_results_copy['data'] >= df_results_copy['data_admissao']) & (df_results_copy['data'] <= df_results_copy['data_demissao'])]
df_results_final = pd.merge(df_results_final, df_colaborador, left_on='colaborador', right_on='matricula', how='left')
df_results_final = df_results_final[(df_results_final['data'] >= df_results_final['data_admissao']) & (df_results_final['data'] <= df_results_final['data_demissao'])]

# Add a column with the number of days worked
df_results_copy['days_worked'] = np.where(df_results_copy['horario'].isin(['M', 'T']), 1, 0)

# Group by colaborador and week_number and count the number of days worked
df_results_copy = df_results_copy = df_results_copy.groupby(['colaborador', 'week_number'])['days_worked'].sum().reset_index()

# Filter rows where days_worked is greater than 5
df_results_copy = df_results_copy[df_results_copy['days_worked'] > 5]

print(df_results_copy)


     colaborador  week_number  days_worked
19             3           37            7
20             3           38            7
22          5851            1            6
74          7656            1            7
126         7919            1            6
178     80000509            1            7
230     80000937            1            7
386     80001237            1            6
438     80001331            1            7
490     80001366            1            7
542     80001489            1            6
720     80003467            1            7
